# 한 채널의 모든 동영상 이름, 재생시간, 날짜 크롤링하기

각자 수정이 필요한 부분에 **강조** 표시를 해두겠습니다!

## 한번만 실행하면 되는 것!

In [24]:
#셀레늄 라이브러리 불러오기
from selenium import webdriver

#셀레늄에서 html문서가 생성될 때까지 대기하는 함수 불러오기
from selenium.webdriver.support.ui import WebDriverWait

#셀레늄에서 조건을 걸어줄 수 있는 함수 불러오기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [25]:
# 필요한 함수들을 불러줍니다!
import re
import time
from pandas import DataFrame
import pandas as pd
import numpy as np

**파일 경로는 각자 환경에 맞게 수정하기**

In [26]:
# youtube_unique_0717.csv 파일을 불러와주세요!
df = pd.read_csv('C:/Users/김지희/DSL/dataset/youtube_unique_0717.csv', lineterminator='\n')

In [27]:
# 채널 아이디의 고유한 값을 channel_unique 변수로 저장해주세요!
channel_unique = df['channelId'].unique()

In [28]:
# 채널에 대해서 일단 긁어오는 거니까 용량을 많이 차지하는 df는 없애줍시다!!!
del(df)

In [45]:
# 아래의 함수는 스크롤을 가장 최하단까지 내려주는 스크롤입니다.
# 저희의 목표는 한 채널의 모든 동영상의 정보를 다 가져오는 것이기에, 스크롤을 내려야만 그 정보들을 가져올 수 있습니다!
# https://heytech.tistory.com/325 참고하였습니다.
# 이 셀은 그냥 실행시켜주시면 됩니다! 맥락만 이해해주세요.


def scroll():
    try:        
        # 페이지 내 스크롤 높이 받아오기
        last_page_height = browser.execute_script("return document.documentElement.scrollHeight")
        count = 0
        while True:
            # 임의의 페이지 로딩 시간 설정
            # PC환경에 따라 로딩시간 최적화를 통해 scraping 시간 단축 가능
            pause_time = 3
            # 페이지 최하단까지 스크롤
            browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            # 페이지 로딩 대기
            time.sleep(pause_time)
            # 무한 스크롤 동작을 위해 살짝 위로 스크롤(i.e., 페이지를 위로 올렸다가 내리는 제스쳐)
            browser.execute_script("window.scrollTo(0, document.documentElement.scrollHeight-50)")
            time.sleep(pause_time)
            count += 1
            # 페이지 내 스크롤 높이 새롭게 받아오기
            new_page_height = browser.execute_script("return document.documentElement.scrollHeight")
            
            
            # 스크롤을 완료한 경우(더이상 페이지 높이 변화가 없는 경우)
            if count == 1 or new_page_height == last_page_height:
                print("스크롤 완료")
                break
                
            # 스크롤 완료하지 않은 경우, 최하단까지 스크롤
            else:
                last_page_height = new_page_height
            
    except Exception as e:
        print("에러 발생: ", e)

**각자 환경에 맞게 수정해주세요**

In [31]:
# 크롬드라이버 경로 설정 (새로운 창이 열릴 겁니다!)
browser = webdriver.Chrome(r"C:\Users\김지희\Downloads\chromedriver.exe")
browser.maximize_window()

C:\ProgramData\Anaconda3\envs\Crawling\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  


In [57]:
# 여기서 range(처음, 끝), 저장경로만 수정하시면 됩니다!
for j in range(100, 110):
    i = j
    browser.get('https://www.youtube.com/channel/'+channel_unique[i]+'/videos')
    
    scroll()
    
    inside = True


    # t는 t번째 영상을 의미합니다.
    t= 0

    # 아래의 리스트에 재생시간, 동영상제목, videoid, 업로드된날짜 네 가지를 차곡차곡 append를 통해 넣겠습니다.
    timelenlist = []
    titlelist = []
    videoidlist = []
    timewhenlist = []
    viewlist = []

    # 드디어 반복문 안으로 들어왔습니다!
    while(inside):
        t=t+1

        try:
            time.sleep(2) #로딩 시간 고려

            timelen = browser.find_element(by = By.XPATH, value = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(t)+']/div[1]/ytd-thumbnail/a/div[1]/ytd-thumbnail-overlay-time-status-renderer/span').text

            timewhen = browser.find_element(by = By.XPATH, value = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(t)+']/div[1]/div[1]/div[1]/div/div[1]/div[2]/span[2]').text

            views = browser.find_element(by = By.XPATH, value = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+str(t)+']/div[1]/div[1]/div[1]/div/div[1]/div[2]/span[1]').text
            info= browser.find_element(by = By.XPATH, value = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['+ str(t) +']/div[1]/div[1]/div[1]/h3/a')

            title = info.get_attribute('title')
            link = info.get_attribute('href')

            if "shorts/" in link:
                link = link.replace("shorts/", "watch?v=")

            videoid = link[link.find('=')+1:]

            timelenlist.append(timelen)
            titlelist.append(title)
            videoidlist.append(videoid)
            timewhenlist.append(timewhen)
            viewlist.append(views)

        except:
            print('%d개의 영상 정보를 크롤링했습니다.'%(t-1))
            inside=False

    one = pd.DataFrame({'time':timelenlist, 'title':titlelist, 'video_id':videoidlist, 'when':timewhenlist, 'views':viewlist}) #괄호 안을 채워주세용!

    one['channelId'] = channel_unique[i]
    youtuber = browser.find_element(by= By.XPATH, value = '/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/div[3]/ytd-c4-tabbed-header-renderer/tp-yt-app-header-layout/div/tp-yt-app-header/div[2]/div[2]/div/div[1]/div/div[1]').text
    one['subscriber'] = youtuber
    
    # 크롤링한 결과를 csv 파일로 합칠 때 채널명 연동을 해야하는데 그 부분을 아직 구현 못해서 일단 unique 인덱스로 해놨습니다.
    one.to_csv('C:/Users/김지희/DSL/dataset/'+str(i)+'_0724.csv', index = False)
    df = pd.read_csv('C:/Users/김지희/DSL/dataset/'+str(i)+'_0724.csv')
    
    # 결과물 확인위해서 head로 출력한 것 => 제거 가능
    print(df.head())

# 에러 발생 시 처리는 구현하지 못했습니다...
    
print('크롤링이 완료되었습니다.')

30개의 영상 정보를 크롤링했습니다.
    time                                              title     video_id  \
0   9:56  공개 잠행💥당권 경쟁💥또또또 인사논란 까지!💥 쓴 소리 지대로다아~~ [KBS 22...  LfAI4PD6v2Y   
1   5:29  통합이 목표라면서 친명-반명??..당내 경쟁자들끼리 '사법리스크' 말하는 거, 이게...  DpJ-rlU1X7E   
2   7:50  사적채용 논란 사과했지만 반응은..."또 갖다 붙인다!! 민주당은 맨날 그래요"vs...  K0k7yq1qHV0   
3  41:40  [더라이브] 533회 풀영상 2022. 7. 21. (임경빈/천하람/김성회) [KB...  LhBfDoeLIT0   
4  42:14  [더라이브] 532회 풀영상 2022. 7. 20. (김수민/정철진/박주민) [KB...  r5iikdr2R1U   

            when      views                 channelId           subscriber  
0         21시간 전  조회수 8.4천회  UCjg8YuDzpuQ5dpWQB7ekWGA  KBS더라이브\n구독자 31.2만명  
1           1일 전  조회수 3.5천회  UCjg8YuDzpuQ5dpWQB7ekWGA  KBS더라이브\n구독자 31.2만명  
2           1일 전    조회수 1만회  UCjg8YuDzpuQ5dpWQB7ekWGA  KBS더라이브\n구독자 31.2만명  
3  스트리밍 시간: 2일 전  조회수 2.8만회  UCjg8YuDzpuQ5dpWQB7ekWGA  KBS더라이브\n구독자 31.2만명  
4  스트리밍 시간: 3일 전  조회수 1.8만회  UCjg8YuDzpuQ5dpWQB7ekWGA  KBS더라이브\n구독자 31.2만명  
30개의 영상 정보를 크롤링했습니다.
    time                               

30개의 영상 정보를 크롤링했습니다.
   time                                              title     video_id  \
0  3:49  [오늘 이 뉴스] "대기발령 오히려 잘됐다" 류삼영 총경 작심 인터뷰(2022.07...  p2WCWSk0xOY   
1  0:54      [날씨] 전국 비 오락가락‥낮 동안 후텁지근 (2022.07.24/12MBC뉴스)  R0Cg7tBfLiI   
2  0:32           내일부터 윤석열 정부 첫 대정부질문 (2022.07.24/12MBC뉴스)  g1XfYTpni5Y   
3  0:52   고의 교통사고로 5억 원대 합의금 챙긴 일당 검거 (2022.07.24/12MBC뉴스)  MtAPJNjkUBU   
4  0:53  美 요세미티공원 인근서 또 대형 산불‥6천여 명 대피령 (2022.07.24/12M...  6n_XKrVMutk   

    when      views                 channelId          subscriber  
0  2시간 전   조회수 52만회  UCF4Wxdo3inmxP-Y59wXDsFw  MBCNEWS\n구독자 237만명  
1  5시간 전  조회수 8.5천회  UCF4Wxdo3inmxP-Y59wXDsFw  MBCNEWS\n구독자 237만명  
2  6시간 전  조회수 2.7만회  UCF4Wxdo3inmxP-Y59wXDsFw  MBCNEWS\n구독자 237만명  
3  6시간 전  조회수 7.5만회  UCF4Wxdo3inmxP-Y59wXDsFw  MBCNEWS\n구독자 237만명  
4  6시간 전  조회수 4.4천회  UCF4Wxdo3inmxP-Y59wXDsFw  MBCNEWS\n구독자 237만명  
